<a href="https://colab.research.google.com/github/s4ngi/ISYS2001/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import gradio as gr
from hands_on_ai.chat import get_response

# ============================================
# ✅ CSV Handling and Analysis
# ============================================

def load_and_clean_csv(file):
    try:
        df = pd.read_csv(file.name)
        expected_columns = ["Date", "Amount", "Category", "Description"]
        if list(df.columns) != expected_columns:
            return "Error: CSV must have columns in this exact order: Date, Amount, Category, Description"
        df["Amount"] = df["Amount"].replace('[\$,]', '', regex=True).astype(float)
        df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce")
        df = df.dropna(subset=["Date", "Amount"])
        if df.empty:
            return "Error: CSV contains no valid data."
        return df
    except Exception as e:
        return f"Error loading CSV: {str(e)}"

def summarize_expenses(df):
    total = df["Amount"].sum()
    category_totals = df.groupby("Category")["Amount"].sum().round(2).to_dict()
    category_averages = df.groupby("Category")["Amount"].mean().round(2).to_dict()
    return {
        "Total Spending": round(total, 2),
        "Category Totals": category_totals,
        "Category Averages": category_averages
    }

def monthly_summary(df):
    return {
        "Monthly Total": round(df["Amount"].sum(), 2),
        "Monthly Average": round(df["Amount"].mean(), 2)
    }

def generate_spending_advice(summary_dict):
    total = summary_dict.get("Total Spending", 0)
    category_totals = summary_dict.get("Category Totals", {})
    if total == 0 or not category_totals:
        return ["No spending data available."]
    advice = []
    for category, amount in category_totals.items():
        percent = (amount / total) * 100
        if percent >= 20:
            advice.append(f"{category} is {percent:.1f}% of your spending. Consider setting a limit.")
        elif percent >= 10:
            advice.append(f"{category} makes up {percent:.1f}%. Keep an eye on it.")
        else:
            advice.append(f"{category} is only {percent:.1f}%. No action needed.")
    return advice

def generate_financial_summary(df):
    try:
        total_expenses = df["Amount"].sum()
        category_totals = df.groupby('Category')["Amount"].sum().round(2).to_dict()
        summary = f"""
- Total Expenses: ${total_expenses:.2f}
Spending by Category:
{pd.Series(category_totals).to_string()}
"""
        advice_lines = generate_spending_advice({
            "Total Spending": total_expenses,
            "Category Totals": category_totals
        })
        summary += "\n\nSpending Advice:\n" + "\n".join(f"- {line}" for line in advice_lines)
        return summary
    except Exception as e:
        return f"Error generating financial summary: {str(e)}"

# --------------------------------------------
# ✅ Table formatting for professional look
# --------------------------------------------
def format_summary_table(df):
    summary = summarize_expenses(df)
    total = summary["Total Spending"]
    table_data = []
    for category, total_amount in summary["Category Totals"].items():
        avg_amount = summary["Category Averages"][category]
        percent = (total_amount / total) * 100
        # Highlight high spending
        highlight = "⚠️" if percent >= 20 else ""
        table_data.append([category, f"${total_amount:.2f} {highlight}", f"${avg_amount:.2f}"])
    return table_data

def format_monthly_table(df):
    monthly = monthly_summary(df)
    return [[f"${monthly['Monthly Total']:.2f}", f"${monthly['Monthly Average']:.2f}"]]

# ============================================
# 💬 Financial Sage Chatbot
# ============================================

def setup_financial_rag(df=None, document_texts=[]):
    """
    Set up RAG system using transaction CSVs and optional financial documents.
    Returns a RetrievalQA chain.
    """
    if df is None and not document_texts:
        return None  # Nothing to retrieve from

    # 1️⃣ Convert CSV to text
    texts = []
    if df is not None:
        texts.append(df.to_string(index=False))

    # 2️⃣ Include additional financial documents
    texts.extend(document_texts)

    # 3️⃣ Split long texts
    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = []
    for t in texts:
        docs.extend(splitter.split_text(t))

    # 4️⃣ Create embeddings
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(docs, embeddings)

    # 5️⃣ Setup retrieval + GPT chain
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    llm = OpenAI(temperature=0)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    return qa_chain

def financial_sage(question: str):
    personality = """
    You are the Financial Sage — a warm, insightful, and practical guide
    who helps users understand money management, budgeting, and saving.
    You sound friendly, calm, and clear — like a patient teacher.
    You use simple examples relevant to students or young adults.
    You never give professional or investment advice — only educational explanations.
    """
    prompt = f"{personality}\nUser question: {question}\nRespond as the Financial Sage."
    return get_response(prompt=prompt)

def get_sage_response(question, df_state):
    """
    Return Financial Sage response. Uses RAG if CSV/docs are available.
    Otherwise, fallback to normal chatbot.
    """
    # Optional financial guides
    guides = [
        "Budgeting guide: Track spending, set limits, save 20% monthly.",
        "Financial advice: Avoid overspending on entertainment, prioritize essentials."
    ]

    # Only setup RAG if there is CSV or guides
    if df_state is not None or guides:
        try:
            # Setup RAG
            rag_chain = setup_financial_rag(df=df_state, document_texts=guides)
            # Use RAG to answer question
            if rag_chain:
                answer = rag_chain.run(question)
                # If RAG fails or returns nothing, fallback to normal
                if not answer.strip():
                    return financial_sage(question)
                return answer
        except Exception as e:
            # On any RAG error, fallback to normal
            print(f"RAG error: {e}")
            return financial_sage(question)
    # No CSV/docs → normal chatbot
    return financial_sage(question)

# ============================================
# 🧩 Gradio App Layout
# ============================================

with gr.Blocks(title="💰 Financial Sage Dashboard", theme=gr.themes.Default(primary_hue="blue")) as app:
    # Header
    gr.Markdown(
        """
        <div style="text-align:center; padding: 10px 0;">
            <h1>💰 Financial Sage Dashboard</h1>
            <p style="font-size:16px; color: gray;">
                Analyze your spending, get smart advice, and chat with the Financial Sage.
            </p>
        </div>
        """
    )

    # State to store uploaded CSV
    df_state = gr.State(value=None)

    # Main layout: left = dashboard, right = chatbot
    with gr.Row():
        # -----------------------
        # Dashboard
        # -----------------------
        with gr.Column(scale=1, min_width=450):
            gr.Markdown("### 📁 Upload Your CSV")
            file_input = gr.File(
                label="Upload your expenses (Date, Amount, Category, Description)",
                file_types=[".csv"]
            )

            gr.Markdown("### 🧾 Financial Summary")
            summary_output = gr.Dataframe(
                headers=["Category", "Total Spending", "Average Spending"],
                label="📂 Category Summary",
                interactive=False
            )
            monthly_output = gr.Dataframe(
                headers=["Monthly Total", "Monthly Average"],
                label="📅 Monthly Summary",
                interactive=False
            )
            advice_output = gr.Textbox(
                label="💡 Spending Insights",
                lines=6,
                interactive=False,
                show_copy_button=True
            )

            file_input.change(
                fn=lambda file: (
                    format_summary_table(load_and_clean_csv(file)),
                    format_monthly_table(load_and_clean_csv(file)),
                    "\n".join(generate_spending_advice(summarize_expenses(load_and_clean_csv(file)))),
                    generate_financial_summary(load_and_clean_csv(file)),
                    load_and_clean_csv(file)
                ),
                inputs=file_input,
                outputs=[summary_output, monthly_output, advice_output, summary_text_output, df_state]
            )

        # -----------------------
        # Chatbot
        # -----------------------
        with gr.Column(scale=1, min_width=400):
            gr.Markdown(
                """
                <div style="padding-bottom:10px;">
                    <h3>💬 Financial Sage Chat</h3>
                    <p style="color: gray;">Ask questions about saving, budgeting, or spending wisely.
                    Upload a CSV in the left panel for data-driven answers.</p>
                </div>
                """
            )
            question_input = gr.Textbox(
                label="Your Question",
                placeholder="e.g. How can I lower my dining expenses?",
                lines=2
            )
            chat_button = gr.Button("Ask the Sage 💭", variant="primary", size="lg")
            chat_output = gr.Textbox(
                label="Financial Sage Response",
                lines=15,
                show_copy_button=True
            )

            chat_button.click(
                fn=get_sage_response,
                inputs=[question_input, df_state],
                outputs=chat_output
            )

    # Footer
    gr.Markdown(
        """
        <hr>
        <p style="text-align:center; color: gray;">
        © 2025 Financial Sage | Educational financial insights for students and beginners.
        </p>
        """
    )

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://724b1825ec25cbb088.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
import pandas as pd
import gradio as gr
from hands_on_ai.chat import get_response

# ============================================
# ✅ CSV Handling and Analysis
# ============================================

def load_and_clean_csv(file):
    try:
        df = pd.read_csv(file.name)
        expected_columns = ["Date", "Amount", "Category", "Description"]
        if list(df.columns) != expected_columns:
            return "Error: CSV must have columns in this exact order: Date, Amount, Category, Description"
        df["Amount"] = df["Amount"].replace('[\$,]', '', regex=True).astype(float)
        df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce")
        df = df.dropna(subset=["Date", "Amount"])
        if df.empty:
            return "Error: CSV contains no valid data."
        return df
    except Exception as e:
        return f"Error loading CSV: {str(e)}"

def summarize_expenses(df):
    total = df["Amount"].sum()
    category_totals = df.groupby("Category")["Amount"].sum().round(2).to_dict()
    category_averages = df.groupby("Category")["Amount"].mean().round(2).to_dict()
    return {
        "Total Spending": round(total, 2),
        "Category Totals": category_totals,
        "Category Averages": category_averages
    }

def monthly_summary(df):
    return {
        "Monthly Total": round(df["Amount"].sum(), 2),
        "Monthly Average": round(df["Amount"].mean(), 2)
    }

def generate_spending_advice(summary_dict):
    total = summary_dict.get("Total Spending", 0)
    category_totals = summary_dict.get("Category Totals", {})
    if total == 0 or not category_totals:
        return ["No spending data available."]
    advice = []
    for category, amount in category_totals.items():
        percent = (amount / total) * 100
        if percent >= 20:
            advice.append(f"{category} is {percent:.1f}% of your spending. Consider setting a limit.")
        elif percent >= 10:
            advice.append(f"{category} makes up {percent:.1f}%. Keep an eye on it.")
        else:
            advice.append(f"{category} is only {percent:.1f}%. No action needed.")
    return advice

def generate_financial_summary(df):
    try:
        total_expenses = df["Amount"].sum()
        category_totals = df.groupby('Category')["Amount"].sum().round(2).to_dict()
        summary = f"""
- Total Expenses: ${total_expenses:.2f}
Spending by Category:
{pd.Series(category_totals).to_string()}
"""
        advice_lines = generate_spending_advice({
            "Total Spending": total_expenses,
            "Category Totals": category_totals
        })
        summary += "\n\nSpending Advice:\n" + "\n".join(f"- {line}" for line in advice_lines)
        return summary
    except Exception as e:
        return f"Error generating financial summary: {str(e)}"

# --------------------------------------------
# ✅ Table formatting for professional look
# --------------------------------------------
def format_summary_table(df):
    summary = summarize_expenses(df)
    total = summary["Total Spending"]
    table_data = []
    for category, total_amount in summary["Category Totals"].items():
        avg_amount = summary["Category Averages"][category]
        percent = (total_amount / total) * 100
        highlight = "⚠️" if percent >= 20 else ""
        table_data.append([category, f"${total_amount:.2f} {highlight}", f"${avg_amount:.2f}"])
    return table_data

def format_monthly_table(df):
    monthly = monthly_summary(df)
    return [[f"${monthly['Monthly Total']:.2f}", f"${monthly['Monthly Average']:.2f}"]]

# ============================================
# 💬 Financial Sage Chatbot
# ============================================

# RAG setup imports
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI as OpenAI
from langchain.chains import RetrievalQA

def setup_financial_rag(df=None, document_texts=[]):
    if df is None and not document_texts:
        return None

    texts = []
    if df is not None:
        texts.append(df.to_string(index=False))
    texts.extend(document_texts)

    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = []
    for t in texts:
        docs.extend(splitter.split_text(t))

    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(docs, embeddings)

    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    llm = OpenAI(temperature=0)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    return qa_chain

def financial_sage(question: str):
    personality = """
    You are the Financial Sage — a warm, insightful, and practical guide
    who helps users understand money management, budgeting, and saving.
    You sound friendly, calm, and clear — like a patient teacher.
    You use simple examples relevant to students or young adults.
    You never give professional or investment advice — only educational explanations.
    """
    prompt = f"{personality}\nUser question: {question}\nRespond as the Financial Sage."
    return get_response(prompt=prompt)

def get_sage_response(question, rag_chain):
    if rag_chain:
        try:
            answer = rag_chain.run(question)
            if answer.strip():
                return answer
        except Exception as e:
            print(f"RAG error: {e}")
    return financial_sage(question)

# ============================================
# 🧩 Gradio App Layout
# ============================================

with gr.Blocks(title="💰 Financial Sage Dashboard", theme=gr.themes.Default(primary_hue="blue")) as app:
    # Header
    gr.Markdown(
        """
        <div style="text-align:center; padding: 10px 0;">
            <h1>💰 Financial Sage Dashboard</h1>
            <p style="font-size:16px; color: gray;">
                Analyze your spending, get smart advice, and chat with the Financial Sage.
            </p>
        </div>
        """
    )

    # States
    df_state = gr.State(value=None)
    rag_chain_state = gr.State(value=None)

    # Layout: Dashboard left, Chatbot right
    with gr.Row():
        # -----------------------
        # Dashboard
        # -----------------------
        with gr.Column(scale=1, min_width=450):
            gr.Markdown("### 📁 Upload Your CSV")
            file_input = gr.File(
                label="Upload your expenses (Date, Amount, Category, Description)",
                file_types=[".csv"]
            )

            gr.Markdown("### 🧾 Financial Summary")
            summary_output = gr.Dataframe(
                headers=["Category", "Total Spending", "Average Spending"],
                label="📂 Category Summary",
                interactive=False
            )
            monthly_output = gr.Dataframe(
                headers=["Monthly Total", "Monthly Average"],
                label="📅 Monthly Summary",
                interactive=False
            )
            advice_output = gr.Textbox(
                label="💡 Spending Insights",
                lines=6,
                interactive=False,
                show_copy_button=True
            )
            summary_text_output = gr.Textbox(
                label="📄 Detailed Summary",
                lines=8,
                interactive=False,
                show_copy_button=True
            )

            # CSV upload processing
            def process_file(file):
                df = load_and_clean_csv(file)
                if isinstance(df, str):
                    return None, None, None, df, None, None
                guides = [
                    "Budgeting guide: Track spending, set limits, save 20% monthly.",
                    "Financial advice: Avoid overspending on entertainment, prioritize essentials."
                ]
                try:
                    rag_chain = setup_financial_rag(df=df, document_texts=guides)
                except Exception as e:
                    print(f"RAG setup error: {e}")
                    rag_chain = None
                return (
                    format_summary_table(df),
                    format_monthly_table(df),
                    "\n".join(generate_spending_advice(summarize_expenses(df))),
                    generate_financial_summary(df),
                    df,
                    rag_chain
                )

            file_input.change(
                fn=process_file,
                inputs=file_input,
                outputs=[summary_output, monthly_output, advice_output, summary_text_output, df_state, rag_chain_state]
            )

        # -----------------------
        # Chatbot
        # -----------------------
        with gr.Column(scale=1, min_width=400):
            gr.Markdown(
                """
                <div style="padding-bottom:10px;">
                    <h3>💬 Financial Sage Chat</h3>
                    <p style="color: gray;">Ask questions about saving, budgeting, or spending wisely.
                    Upload a CSV in the left panel for data-driven answers.</p>
                </div>
                """
            )
            question_input = gr.Textbox(
                label="Your Question",
                placeholder="e.g. How can I lower my dining expenses?",
                lines=2
            )
            chat_button = gr.Button("Ask the Sage 💭", variant="primary", size="lg")
            chat_output = gr.Textbox(
                label="Financial Sage Response",
                lines=15,
                show_copy_button=True
            )

            chat_button.click(
                fn=get_sage_response,
                inputs=[question_input, rag_chain_state],
                outputs=chat_output
            )

    # Footer
    gr.Markdown(
        """
        <hr>
        <p style="text-align:center; color: gray;">
        © 2025 Financial Sage | Educational financial insights for students and beginners.
        </p>
        """
    )

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d140851a6c07d77bc6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
import pandas as pd
import gradio as gr
from hands_on_ai.chat import get_response
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter

# ---------------------------
# CSV Handling and Analysis
# ---------------------------
def load_and_clean_csv(file):
    try:
        df = pd.read_csv(file.name)
        expected_columns = ["Date", "Amount", "Category", "Description"]
        if list(df.columns) != expected_columns:
            return None, "Error: CSV must have columns: Date, Amount, Category, Description"
        df["Amount"] = df["Amount"].replace('[\$,]', '', regex=True).astype(float)
        df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce")
        df = df.dropna(subset=["Date", "Amount"])
        if df.empty:
            return None, "Error: CSV contains no valid data."
        return df, None
    except Exception as e:
        return None, f"Error loading CSV: {str(e)}"

def summarize_expenses(df):
    total = df["Amount"].sum()
    category_totals = df.groupby("Category")["Amount"].sum().round(2).to_dict()
    category_averages = df.groupby("Category")["Amount"].mean().round(2).to_dict()
    return {
        "Total Spending": round(total, 2),
        "Category Totals": category_totals,
        "Category Averages": category_averages
    }

def monthly_summary(df):
    return {
        "Monthly Total": round(df["Amount"].sum(), 2),
        "Monthly Average": round(df["Amount"].mean(), 2)
    }

def generate_spending_advice(summary_dict):
    total = summary_dict.get("Total Spending", 0)
    category_totals = summary_dict.get("Category Totals", {})
    if total == 0 or not category_totals:
        return ["No spending data available."]
    advice = []
    for category, amount in category_totals.items():
        percent = (amount / total) * 100
        if percent >= 20:
            advice.append(f"{category} is {percent:.1f}% of your spending. Consider setting a limit.")
        elif percent >= 10:
            advice.append(f"{category} makes up {percent:.1f}%. Keep an eye on it.")
        else:
            advice.append(f"{category} is only {percent:.1f}%. No action needed.")
    return advice

def format_summary_table(df):
    summary = summarize_expenses(df)
    total = summary["Total Spending"]
    table_data = []
    for category, total_amount in summary["Category Totals"].items():
        avg_amount = summary["Category Averages"][category]
        percent = (total_amount / total) * 100
        highlight = "⚠️" if percent >= 20 else ""
        table_data.append([category, f"${total_amount:.2f} {highlight}", f"${avg_amount:.2f}"])
    return table_data

def format_monthly_table(df):
    monthly = monthly_summary(df)
    return [[f"${monthly['Monthly Total']:.2f}", f"${monthly['Monthly Average']:.2f}"]]

# ---------------------------
# RAG Chatbot Setup
# ---------------------------
def setup_financial_rag(df):
    """
    Convert CSV to RAG retriever for chatbot
    """
    texts = [df.to_string(index=False)]
    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = []
    for t in texts:
        docs.extend(splitter.split_text(t))
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(docs, embeddings)
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    llm = OpenAI(temperature=0)
    return RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

def financial_sage(question: str):
    personality = """
You are the Financial Sage — a warm, insightful, and practical guide
who helps users understand money management, budgeting, and saving.
You sound friendly, calm, and clear — like a patient teacher.
You never give professional or investment advice — only educational explanations.
"""
    prompt = f"{personality}\nUser question: {question}\nRespond as the Financial Sage."
    return get_response(prompt=prompt)

def get_sage_response(question, df_state, rag_cache={}):
    """
    Use CSV in df_state to answer via RAG. Cache vector store for speed.
    """
    if df_state is not None:
        if "rag_chain" not in rag_cache:
            rag_cache["rag_chain"] = setup_financial_rag(df_state)
        try:
            answer = rag_cache["rag_chain"].run(question)
            if answer.strip():
                return answer
        except:
            pass
    # fallback
    return financial_sage(question)

# ---------------------------
# Gradio App
# ---------------------------
with gr.Blocks(title="💰 Financial Sage Dashboard") as app:
    df_state = gr.State(value=None)
    rag_cache = gr.State(value={})

    with gr.Row():
        # Dashboard
        with gr.Column(scale=1, min_width=450):
            gr.Markdown("### 📁 Upload Your CSV")
            file_input = gr.File(label="Upload expenses CSV", file_types=[".csv"])
            summary_output = gr.Dataframe(headers=["Category", "Total Spending", "Average Spending"], interactive=False)
            monthly_output = gr.Dataframe(headers=["Monthly Total", "Monthly Average"], interactive=False)
            advice_output = gr.Textbox(label="💡 Spending Insights", lines=6, interactive=False, show_copy_button=True)

            def handle_file_upload(file):
                df, error = load_and_clean_csv(file)
                if error:
                    return [], [], error, None
                return (
                    format_summary_table(df),
                    format_monthly_table(df),
                    "\n".join(generate_spending_advice(summarize_expenses(df))),
                    df
                )

            file_input.change(
                fn=handle_file_upload,
                inputs=file_input,
                outputs=[summary_output, monthly_output, advice_output, df_state]
            )

        # Chatbot
        with gr.Column(scale=1, min_width=400):
            gr.Markdown("### 💬 Financial Sage Chat")
            question_input = gr.Textbox(label="Your Question", placeholder="e.g. How can I lower my dining expenses?", lines=2)
            chat_button = gr.Button("Ask the Sage 💭")
            chat_output = gr.Textbox(label="Financial Sage Response", lines=15, show_copy_button=True)

            chat_button.click(fn=get_sage_response, inputs=[question_input, df_state, rag_cache], outputs=chat_output)

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://85f5b2d372efe235b3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
# Installing required packages
!pip install pandas gradio hands_on_ai python-dateutil langchain openai faiss-cpu
# Import core libraries
import pandas as pd
import gradio as gr
import os
import hands_on_ai as ha
from datetime import datetime
from hands_on_ai.chat import get_response
from dateutil import parser
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter

import warnings
warnings.filterwarnings('ignore')

print("Core libraries loaded successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"Gradio version: {gr.__version__}")
print(f"OS version: {os.name}")
print(f"Hands-on-AI version: {ha.__version__}")
print("All libraries loaded successfully!")

from getpass import getpass

# Configure hands-on-ai server connection
os.environ['HANDS_ON_AI_SERVER'] = 'https://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = 'isys2001-assignment-key'

print("Hands-on-AI configured successfully!")


Core libraries loaded successfully!
Pandas version: 2.2.2
Gradio version: 5.49.1
OS version: posix
Hands-on-AI version: 0.1.10
All libraries loaded successfully!
Hands-on-AI configured successfully!
